In [8]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
np.random.seed(42)
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer



from csv import reader


with open('cs305fp_music_genre.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    count = 0
    data = []
    titles = []
    artists = []
    instance_id = []
    genres = []
    musicKeys = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']
    
    for row in csv_reader:
        if count == 0:
            columns = row
        else:
            instance_id.append(row[0])
            artists.append(row[1])
            titles.append(row[2])
            
            #music key (9 in list)
            if row[9] in musicKeys:
                row[9] = musicKeys.index(row[9])
            
            #mode (12 in list)
            if row[12] == 'Major':
                row[12] = 1
            elif row[12] == 'Minor':
                row[12] = 0
            
            
            #tempo (14 in list) 
            if row[14] == "?":
                row[14] = ''
            
            #music_genre (17 in list)
            if row[17] in genres:
                row[17] = genres.index(row[17])
            else:
                genres.append(row[17])
                row[17] = genres.index(row[17])
            
            #make sure to not include list items 0-2 and item 15 in list
            new = row[3:15]+row[16:]
            data.append(new)
            
        count = count + 1
print(data[1])

data = np.array(data)
        
        
            

['31.0', '0.0127', '0.622', '218293.0', '0.89', '0.95', 2, '0.124', '-7.042999999999999', 0, '0.03', '115.00200000000001', '0.531', 0]
